In [2]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision torchaudio

     ---------------------------------------- 3.7/3.7 MB 9.1 MB/s eta 0:00:00
     ------------------------------------- 663.9/663.9 kB 10.5 MB/s eta 0:00:00
     -------------------------------------- 111.0/111.0 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 33.9 MB/s eta 0:00:00


In [24]:
import os
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import global_add_pool
from torch_cluster import radius_graph
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem, rdchem
import matplotlib.pyplot as plt
import math

환경결정상수

In [25]:
# H5 파일 키 설정
KEY_CONFIG = {
    "atoms": "elements", "smiles": "Rsmiles",
    "R_coords": "RG", "TS_coords": "TSG", "P_coords": "PG",          
    "R_energy": { "H": "R_H", "G": "R_F", "E": "R_E" },
    "TS_energy": { "H": "TS_H", "G": "TS_F", "E": "TS_E" },
    "P_energy": { "H": "P_H", "G": "P_F", "E": "P_E" }
}

# 결합 해리 에너지 (BDE) 테이블 (kcal/mol)
BDE_TABLE = {
    (1, 1): 104, (1, 6): 99, (1, 7): 93, (1, 8): 111, (1, 9): 135,
    (6, 6): 83, (6, 7): 73, (6, 8): 86, (6, 9): 116,
    (7, 7): 39, (7, 8): 53, (8, 8): 35
}

# 원자 물성 [EN, Hardness, Radius, IE]
ATOM_PROPS_TABLE = {
    1: [2.20, 6.42, 0.31, 13.60], 6: [2.55, 5.00, 0.76, 11.26],
    7: [3.04, 7.27, 0.71, 14.53], 8: [3.44, 6.08, 0.66, 13.62], 9: [3.98, 7.01, 0.57, 17.42]
}
DEFAULT_PROPS = [0.0, 0.0, 0.0, 0.0]

# 작용기 SMARTS 패턴
FUNCTIONAL_GROUPS = {
    "Nitro": "[N+](=O)[O-]", "Sulfonyl": "S(=O)(=O)", "Cyano": "C#N", "Trifluoromethyl": "C(F)(F)F",
    "Carbonyl": "[CX3]=[OX1]", "Carboxyl": "[CX3](=O)[OX2H1]", "Halogen": "[F,Cl,Br,I]",
    "Amine": "[NX3;H2,H1,H0;!$(NC=O)]", "Hydroxy": "[OX2H]", "Alkoxy": "[OX2][CX4]", 
    "Amide": "[NX3][CX3](=O)", "Alkyl": "[CX4]", "Phenyl": "c1ccccc1"
}
FG_TYPE_MAP = {
    "Nitro": -1.0, "Sulfonyl": -1.0, "Cyano": -1.0, "Trifluoromethyl": -1.0,
    "Carbonyl": -0.5, "Carboxyl": -0.5, "Halogen": -0.2, "Amine": 1.0, "Hydroxy": 1.0, 
    "Alkoxy": 1.0, "Amide": 0.2, "Alkyl": 0.3, "Phenyl": 0.1
}

데이터셋 (T1x)

In [26]:
class Transition1xDataset(Dataset):
    def __init__(self, h5_path, limit=10000):
        super().__init__(None, None)
        self.data_list = []
        self.limit = limit
        
        # RGD1과 동일한 Feature 생성을 위한 매핑 테이블
        self.sym_map = {1: 0, 6: 1, 7: 2, 8: 3, 9: 4} 
        self.mass_map = {1: 1.008, 6: 12.011, 7: 14.007, 8: 15.999, 9: 18.998}
        
        self.load_data(h5_path)

    def get_dummy_features(self, atomic_numbers):
        """
        원자 번호를 받아 RGD1 모델과 호환되는 27차원 Dummy Feature를 생성합니다.
        """
        num_atoms = len(atomic_numbers)
        features = torch.zeros((num_atoms, 27), dtype=torch.float)
        
        for i, z_val in enumerate(atomic_numbers):
            z = int(z_val.item())
            idx = self.sym_map.get(z, 5) 
            if idx < 6:
                features[i, idx] = 1.0
            mass = self.mass_map.get(z, 12.0)
            features[i, 14] = mass / 20.0
            
        return features

    def load_data(self, h5_path):
        print(f"Loading Transition1x Data from {h5_path} (Limit: {self.limit})...")
        
        try:
            with h5py.File(h5_path, "r") as f:
                if 'data' not in f:
                    print("Error: 'data' key not found in h5 file.")
                    return
                
                split = f['data']
                count = 0
                
                for formula in tqdm(split.keys(), desc="Parsing T1x"):
                    if count >= self.limit: break
                    
                    grp = split[formula]
                    for rxn in grp.keys():
                        if count >= self.limit: break
                        
                        subgrp = grp[rxn]
                        try:
                            z = torch.tensor(subgrp['atomic_numbers'], dtype=torch.long)
                            x_feat = self.get_dummy_features(z)
                            
                            pos_R = torch.tensor(subgrp['reactant']['positions'][-1], dtype=torch.float)
                            pos_P = torch.tensor(subgrp['product']['positions'][-1], dtype=torch.float)
                            pos_TS = torch.tensor(subgrp['transition_state']['positions'][-1], dtype=torch.float)
                            
                            data = Data(x=x_feat, z=z, pos=pos_R, y_pos=pos_TS, pos_P=pos_P)
                            
                            num_nodes = z.size(0)
                            data.edge_index = radius_graph(pos_R, r=5.0, max_num_neighbors=50)
                            num_edges = data.edge_index.size(1)
                            
                            data.hsab = torch.zeros((num_nodes, 2), dtype=torch.float) 
                            data.edge_bde = torch.zeros((num_edges, 1), dtype=torch.float)
                            
                            # [핵심 수정] Shape를 (1, 1) -> (1, 3)으로 변경 (H, G, E 채널 맞춤)
                            data.y_fwd = torch.zeros((1,1), dtype=torch.float)
                            data.y_rev = torch.zeros((1,1), dtype=torch.float)
                            data.thermo_energy = torch.zeros((1, 3), dtype=torch.float) 
                            
                            self.data_list.append(data)
                            count += 1
                            
                        except Exception as e:
                            continue
                            
        except Exception as e:
            print(f"File Load Error: {e}")
        
        print(f"Loaded {len(self.data_list)} reactions for Pre-training.")

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

데이터셋 (RGD1)

In [27]:
class ReactionDataset(Dataset):
    def __init__(self, h5_path, limit=None):
        super().__init__(None, transform=None, pre_transform=None)
        self.h5_path = h5_path
        if not os.path.exists(h5_path): raise FileNotFoundError(f"File not found: {h5_path}")
        with h5py.File(self.h5_path, 'r') as f:
            self.keys = list(f.keys())
            if limit: self.keys = self.keys[:limit]
            self.len_data = len(self.keys)
        self.fg_patterns = {name: Chem.MolFromSmarts(smarts) for name, smarts in FUNCTIONAL_GROUPS.items()}

    def len(self): return self.len_data

    def get_atom_features(self, mol):
        try: AllChem.ComputeGasteigerCharges(mol); 
        except:
            pass
        features, hsab_list = [], []
        
        # 작용기(Functional Group) 탐지
        atom_fg_info = {atom.GetIdx(): {'ewg':0.0, 'edg':0.0, 'types':[]} for atom in mol.GetAtoms()}
        for name, pattern in self.fg_patterns.items():
            if pattern:
                for match in mol.GetSubstructMatches(pattern):
                    effect = FG_TYPE_MAP.get(name, 0.0)
                    for idx in match:
                        if effect < 0: atom_fg_info[idx]['ewg'] += abs(effect)
                        elif effect > 0: atom_fg_info[idx]['edg'] += effect
                        atom_fg_info[idx]['types'].append(name)
        
        atoms_with_map = [(a.GetAtomMapNum() if a.GetAtomMapNum()!=0 else a.GetIdx()+1, a) for a in mol.GetAtoms()]
        atoms_with_map.sort(key=lambda x: x[0])
        
        for _, atom in atoms_with_map:
            feat = []
            z = atom.GetAtomicNum()
            idx = atom.GetIdx()
            
            # [1] Basic Info (21 dims)
            sym_map={'H':0,'C':1,'N':2,'O':3,'F':4}; oh=[0]*6; oh[sym_map.get(atom.GetSymbol(),5)]=1; feat.extend(oh)
            hyb=atom.GetHybridization(); hyb_map={rdchem.HybridizationType.SP:0, rdchem.HybridizationType.SP2:1, rdchem.HybridizationType.SP3:2, rdchem.HybridizationType.SP3D:3, rdchem.HybridizationType.SP3D2:4}; hyb_vec=[0]*5; 
            if hyb in hyb_map: hyb_vec[hyb_map[hyb]]=1
            feat.extend(hyb_vec)
            try: q=float(atom.GetProp('_GasteigerCharge')); 
            except: q=0.0
            if np.isnan(q): q=0.0
            feat.extend([q, float(atom.GetFormalCharge()), float(atom.GetTotalValence()), atom.GetMass()/20.0])
            feat.extend([1.0 if atom.IsInRingSize(s) else 0.0 for s in [3,4,5,6]])
            feat.extend([1.0 if atom.GetIsAromatic() else 0.0, 1.0 if atom.GetChiralTag()!=rdchem.ChiralType.CHI_UNSPECIFIED else 0.0])
            
            # [2] Functional Group Info (6 dims)
            info = atom_fg_info[idx]
            feat.extend([info['ewg'], info['edg'], 1.0 if 'Nitro' in info['types'] or 'Sulfonyl' in info['types'] else 0.0, 1.0 if 'Carbonyl' in info['types'] else 0.0, 1.0 if 'Amine' in info['types'] or 'Hydroxy' in info['types'] else 0.0, 1.0 if 'Halogen' in info['types'] else 0.0])
            features.append(feat)
            
            # [3] HSAB Props
            props = ATOM_PROPS_TABLE.get(z, DEFAULT_PROPS)
            hsab_list.append([-props[0], props[1]]) # [Chemical Potential, Hardness]
            
        return torch.tensor(features, dtype=torch.float), torch.tensor(hsab_list, dtype=torch.float)

    def get(self, idx):
        with h5py.File(self.h5_path, 'r') as f:
            group = f[self.keys[idx]]
            
            # RDKit Parsing
            if KEY_CONFIG["smiles"] in group:
                smi = group[KEY_CONFIG["smiles"]][()]; smi = smi.decode('utf-8') if isinstance(smi, bytes) else str(smi)
                mol = Chem.MolFromSmiles(smi)
                if mol: mol = Chem.AddHs(mol); x, hsab = self.get_atom_features(mol)
                else: atoms = group[KEY_CONFIG["atoms"]][()]; x = torch.zeros(len(atoms), 27); hsab = torch.zeros(len(atoms), 2)
            else: atoms = group[KEY_CONFIG["atoms"]][()]; x = torch.zeros(len(atoms), 27); hsab = torch.zeros(len(atoms), 2)
            
            # Coordinates
            pos_R = torch.tensor(group[KEY_CONFIG["R_coords"]][()], dtype=torch.float)
            pos_TS = torch.tensor(group[KEY_CONFIG["TS_coords"]][()], dtype=torch.float)
            pos_P = torch.tensor(group[KEY_CONFIG["P_coords"]][()], dtype=torch.float)
            
            # Energies
            e_R = torch.tensor([group[KEY_CONFIG["R_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            e_TS = torch.tensor([group[KEY_CONFIG["TS_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            e_P = torch.tensor([group[KEY_CONFIG["P_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            
            # Targets (Delta Learning ready: predicting barriers directly)
            y_fwd = (e_TS - e_R).unsqueeze(0)
            y_rev = (e_TS - e_P).unsqueeze(0)
            thermo = (e_P - e_R).unsqueeze(0)
            
            # Consistency Check
            if x.size(0)!=pos_R.size(0): m=min(x.size(0),pos_R.size(0)); x=x[:m]; hsab=hsab[:m]; pos_R=pos_R[:m]; pos_P=pos_P[:m]; pos_TS=pos_TS[:m]
            
            # Fully Connected Edges
            n=x.size(0); r=torch.arange(n).repeat_interleave(n); c=torch.arange(n).repeat(n); mask=r!=c
            edge_index=torch.stack([r[mask], c[mask]], dim=0)
            
            # BDE Features
            sym=torch.argmax(x[:,:6], dim=1); map_z={0:1,1:6,2:7,3:8,4:9,5:0}; z=torch.tensor([map_z.get(i.item(),0) for i in sym], dtype=torch.long)
            bde = [BDE_TABLE.get(tuple(sorted((int(z[i]), int(z[j])))), 80.0) for i,j in edge_index.T]
            edge_bde = torch.tensor(bde, dtype=torch.float).unsqueeze(-1)
            
            return Data(x=x, z=z, hsab=hsab, pos=pos_R, pos_P=pos_P, edge_index=edge_index, 
                        y_fwd=y_fwd, y_rev=y_rev, y_pos=pos_TS, thermo_energy=thermo, edge_bde=edge_bde)

GNN Backbone

In [28]:
class RBFLayer(nn.Module):
    """
    [Radial Basis Function Layer]
    거리 정보를 단순히 하나의 숫자가 아닌, 여러 개의 기저 함수(Gaussian)로 확장하여 표현합니다.
    이는 Geometric Deep Learning (SchNet, DimeNet)의 핵심 요소입니다.
    """
    def __init__(self, num_rbf=50, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        # 0 ~ cutoff 사이를 num_rbf 개수만큼 등분
        self.centers = nn.Parameter(torch.linspace(0, cutoff, num_rbf), requires_grad=False)
        self.widths = nn.Parameter(torch.tensor([cutoff / num_rbf] * num_rbf), requires_grad=False)

    def forward(self, dist):
        # Gaussian Smearing: exp(- (d - mu)^2 / beta)
        return torch.exp(-((dist - self.centers) ** 2) / self.widths)

In [29]:
class GeometricInteractionLayer(nn.Module):
    """
    [Geometric Interaction Block]
    RBF로 확장된 거리 정보를 이용하여 노드 간의 상호작용을 학습합니다.
    Simple MLP보다 훨씬 풍부한 기하학적 특징을 추출합니다.
    """
    def __init__(self, hidden_dim, num_rbf=50):
        super().__init__()
        self.rbf_lin = nn.Linear(num_rbf, hidden_dim)
        
        # Message Passing MLPs
        self.msg_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.update_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, h, edge_index, rbf_dist):
        row, col = edge_index
        
        # 1. 거리 정보 변환 (RBF -> Hidden)
        edge_geom = self.rbf_lin(rbf_dist) # [E, Hidden]
        
        # 2. 메시지 생성 (Source + Target) * Geometry
        # SchNet 방식의 Continuous Filter Convolution 아이디어 차용
        node_feat = torch.cat([h[row], h[col]], dim=-1)
        msg_raw = self.msg_net(node_feat)
        msg = msg_raw * edge_geom # Element-wise multiplication with Geometry
        
        # 3. 집계 (Aggregation)
        aggr = torch.zeros_like(h)
        aggr.index_add_(0, row, msg)
        
        # 4. 업데이트 (Update)
        update = self.update_net(torch.cat([h, aggr], dim=-1))
        
        return self.norm(h + update)

In [30]:
class GeometricGNN(nn.Module):
    """
    [Main Backbone]
    위상학적 정보뿐만 아니라 기하학적(RBF) 정보를 적극 활용하는 GNN입니다.
    """
    def __init__(self, node_dim, hidden_dim, num_layers=4):
        super().__init__()
        self.node_emb = nn.Linear(node_dim, hidden_dim)
        self.rbf = RBFLayer(num_rbf=50, cutoff=10.0)
        self.layers = nn.ModuleList([
            GeometricInteractionLayer(hidden_dim, num_rbf=50) for _ in range(num_layers)
        ])

    def forward(self, x, pos, edge_index):
        h = self.node_emb(x)
        
        # 거리 계산 및 RBF 변환
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        rbf_feat = self.rbf(dist) # [E, 50]
        
        for layer in self.layers:
            h = layer(h, edge_index, rbf_feat)
            
        return h

In [31]:
class SigmoidGating(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.gate_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 1) 
        )
    def forward(self, h, edge_index, pos):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        # Logits 반환
        return self.gate_mlp(torch.cat([h[row], h[col], dist], dim=-1))

In [32]:
class StructureDecoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.SiLU(), nn.Linear(hidden_dim, 1))
    def forward(self, h, edge_index, gate_score):
        row, col = edge_index
        return self.mlp(torch.cat([h[row], h[col]], dim=-1)) * gate_score

물리법칙

In [33]:
class PaulingBondLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.r0_table = torch.tensor([0.0, 0.31, 0.0, 0.0, 0.0, 0.0, 0.76, 0.71, 0.66, 0.57]) 
    def forward(self, z, edge_index, dist):
        row, col = edge_index
        r0_i = self.r0_table.to(z.device)[z[row]]
        r0_j = self.r0_table.to(z.device)[z[col]]
        r0_sum = (r0_i + r0_j).unsqueeze(-1)
        return torch.exp((r0_sum - dist) / 0.3)

In [34]:
class StereoelectronicLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos_R, pos_P, edge_index):
        row, col = edge_index
        vec_R = F.normalize(pos_R[col] - pos_R[row], p=2, dim=-1)
        vec_P = F.normalize(pos_P[col] - pos_P[row], p=2, dim=-1)
        return (vec_R * vec_P).sum(dim=-1, keepdim=True)

In [35]:
class HSABInteractionLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, hsab, edge_index):
        row, col = edge_index
        mu, eta = hsab[:, 0], hsab[:, 1]
        d_mu = (mu[row] - mu[col]).pow(2)
        sum_eta = eta[row] + eta[col]
        return (d_mu / (sum_eta + 1e-6)).unsqueeze(-1)

In [36]:
class StericLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos, edge_index):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1)
        density = 1.0 / (dist.pow(2) + 0.1)
        steric = torch.zeros(pos.size(0), 1, device=pos.device)
        steric.index_add_(0, row, density.unsqueeze(-1))
        return steric

In [37]:
class HammondGate(nn.Module):
    def __init__(self):
        super().__init__()
        self.gate = nn.Sequential(nn.Linear(3, 16), nn.SiLU(), nn.Linear(16, 1), nn.Sigmoid())
    def forward(self, thermo): return self.gate(thermo)

Diffusion

In [38]:
class SinusoidalTimeEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

In [39]:
class EGNNLayer(nn.Module):
    def __init__(self, hidden_dim, edge_attr_dim=0):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # [수정] edge_attr_dim 추가 (R/P와의 거리 정보 입력용)
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1 + edge_attr_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU()
        )
        
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.coord_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1, bias=False)
        )

    def forward(self, h, pos, edge_index, edge_attr=None):
        row, col = edge_index
        
        # (1) Radial Distance (Current State)
        coord_diff = pos[row] - pos[col]
        radial = torch.sum(coord_diff**2, dim=1, keepdim=True)
        
        # (2) Edge Feature Construction
        # [수정] edge_attr(조건 정보)가 있으면 함께 Concatenate
        edge_feats = [h[row], h[col], radial]
        if edge_attr is not None:
            edge_feats.append(edge_attr)
            
        edge_input = torch.cat(edge_feats, dim=1)
        msg = self.edge_mlp(edge_input)
        
        # (3) Coordinate Update (Equivariant)
        trans = coord_diff * self.coord_mlp(msg)
        agg_trans = torch.zeros_like(pos)
        agg_trans.index_add_(0, row, trans)
        pos_new = pos + agg_trans
        
        # (4) Node Feature Update
        agg_msg = torch.zeros_like(h)
        agg_msg.index_add_(0, row, msg)
        node_input = torch.cat([h, agg_msg], dim=1)
        h_new = h + self.node_mlp(node_input)
        
        return h_new, pos_new

In [40]:
class EquivariantDenoisingNetwork(nn.Module):
    def __init__(self, node_dim, hidden_dim, num_layers=4):
        super().__init__()
        
        # Time Embedding
        self.time_mlp = nn.Sequential(
            SinusoidalTimeEmbeddings(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.emb = nn.Linear(node_dim, hidden_dim)
        
        # [수정] EGNNLayer에 edge_attr_dim=2 추가 (dist_to_R, dist_to_P)
        self.layers = nn.ModuleList([
            EGNNLayer(hidden_dim, edge_attr_dim=2) for _ in range(num_layers)
        ])
        
        self.final_coord_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1, bias=False)
        )

    def forward(self, h_raw, pos_t, t, edge_index, pos_R, pos_P):
        # 1. Embeddings
        h = self.emb(h_raw)
        time_emb = self.time_mlp(t)
        
        # Time Injection (Simple addition for broadcasting)
        # Note: In production, gather time_emb by batch index
        # h = h + time_emb[batch_index] logic assumes implicit broadcasting or mapped externally
        
        # 2. Conditioning Features (Geometry-Awareness)
        # 현재 노이즈 상태(pos_t)가 R과 P로부터 얼마나 떨어져 있는지 계산
        row, col = edge_index
        
        # Edge: Distance to Reactant (relative difference projected on edge?)
        # 더 간단하고 강력한 방법: Node별 거리를 Edge로 확장
        # 여기서는 Edge Feature로 "두 노드 간 거리" 대신 "R/P 구조에서의 거리"를 참고하도록 함
        
        # (A) R/P 구조에서의 Pairwise Distance (Invariant Conditioning)
        dist_R = (pos_R[row] - pos_R[col]).norm(dim=-1, keepdim=True)
        dist_P = (pos_P[row] - pos_P[col]).norm(dim=-1, keepdim=True)
        
        # 이를 Edge Attribute로 사용
        cond_attr = torch.cat([dist_R, dist_P], dim=-1) # [E, 2]
        
        # 3. EGNN Layers
        curr_pos = pos_t
        curr_h = h
        
        for layer in self.layers:
            # Time embedding injection could be added inside layer, 
            # here we assume h already carries some state or we add it:
            # curr_h = curr_h + time_emb_mapped
            curr_h, curr_pos = layer(curr_h, curr_pos, edge_index, edge_attr=cond_attr)
            
        noise_pred = curr_pos - pos_t 
        return noise_pred

In [41]:
class TSDiffusionModule(nn.Module):
    def __init__(self, node_dim, hidden_dim, timesteps=100):
        super().__init__()
        self.timesteps = timesteps
        
        # Schedule
        beta_start, beta_end = 1e-4, 0.02
        betas = torch.linspace(beta_start, beta_end, timesteps)
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.register_buffer('betas', betas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1. - alphas_cumprod))
        
        self.denoise_net = EquivariantDenoisingNetwork(node_dim, hidden_dim)

    def compute_loss(self, true_ts_pos, x_feat, edge_index, batch, pos_R, pos_P):
        batch_size = batch.max().item() + 1
        device = true_ts_pos.device
        
        t = torch.randint(0, self.timesteps, (batch_size,), device=device).long()
        noise = torch.randn_like(true_ts_pos)
        
        t_node = t[batch]
        sqrt_alpha = self.sqrt_alphas_cumprod[t_node].unsqueeze(-1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t_node].unsqueeze(-1)
        
        noisy_pos = sqrt_alpha * true_ts_pos + sqrt_one_minus_alpha * noise
        
        # Predict Noise (Conditioned on R and P)
        noise_pred = self.denoise_net(x_feat, noisy_pos, t, edge_index, pos_R, pos_P)
        
        # --- [1] Active Atom Weighting ---
        # R과 P 사이의 이동 거리가 큰 원자(반응 중심)에 더 큰 가중치 부여
        displacement = (pos_R - pos_P).norm(dim=-1, keepdim=True) # [N, 1]
        # 이동 거리가 0.5A 이상이면 가중치 3.0, 아니면 1.0
        atom_weights = torch.where(displacement > 0.5, 3.0, 1.0)
        
        # Weighted MSE Loss (Coordinate)
        loss_coord = (atom_weights * (noise_pred - noise) ** 2).mean()
        
        # --- [2] Auxiliary Distance Loss ---
        # 예측된 노이즈로 "복원된 구조(Denoised)"의 거리 행렬이 "실제 구조"와 같아야 함
        # x0_pred = (xt - sqrt(1-a)*eps_pred) / sqrt(a)
        pred_x0 = (noisy_pos - sqrt_one_minus_alpha * noise_pred) / (sqrt_alpha + 1e-8)
        
        # Pairwise Distances
        row, col = edge_index
        d_pred = (pred_x0[row] - pred_x0[col]).norm(dim=-1)
        d_true = (true_ts_pos[row] - true_ts_pos[col]).norm(dim=-1)
        
        loss_dist = F.mse_loss(d_pred, d_true)
        
        # Total Structure Loss
        return loss_coord + 0.5 * loss_dist

    @torch.no_grad()
    def sample(self, pos_R, pos_P, x_feat, edge_index, batch):
        # Initialization: Linear Interpolation (Better start than simple midpoint)
        current_pos = (pos_R + pos_P) / 2.0 + torch.randn_like(pos_R) * 0.1
        
        for t in reversed(range(self.timesteps)):
            batch_size = batch.max().item() + 1
            t_batch = torch.full((batch_size,), t, device=pos_R.device, dtype=torch.long)
            
            # Predict
            noise_pred = self.denoise_net(x_feat, current_pos, t_batch, edge_index, pos_R, pos_P)
            
            # Update
            alpha = 1 - self.betas[t]
            alpha_cumprod = self.alphas_cumprod[t]
            sigma = torch.sqrt(self.betas[t])
            
            mean = (1 / torch.sqrt(alpha)) * (
                current_pos - (self.betas[t] / torch.sqrt(1 - alpha_cumprod)) * noise_pred
            )
            
            if t > 0:
                noise = torch.randn_like(current_pos)
                current_pos = mean + sigma * noise
            else:
                current_pos = mean
                
        return current_pos

Predictor

In [42]:
class PhysChemInformedPredictor(nn.Module):
    def __init__(self, hidden_dim, dropout=0.1):
        super().__init__()
        self.pauling = PaulingBondLayer()
        self.stereo = StereoelectronicLayer()
        self.hsab = HSABInteractionLayer()
        self.steric = StericLayer()
        self.hammond = HammondGate()
        
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim + 4, hidden_dim), nn.SiLU(), nn.Dropout(dropout), nn.Linear(hidden_dim, 3)
        )
        # Edge Feats: 2H + Dist(1)*2 + Gate(1) + Pauling(3) + Stereo(1) + HSAB(1) + BDE(1) + Hammond(1) + PredD(1) = 2H + 11
        edge_in = (hidden_dim * 2) + 11
        self.edge_mlp = nn.Sequential(
            nn.Linear(edge_in, hidden_dim), nn.SiLU(), nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 6) # Fwd(3) + Rev(3)
        )

    def forward(self, h, z, hsab, edge_index, pos_R, pos_P, gate_score, pred_delta_D, batch, thermo, bde):
        row, col = edge_index
        late_char = self.hammond(thermo); late_char_edge = late_char[batch[row]]
        
        steric_R = self.steric(pos_R, edge_index)
        steric_graph = global_add_pool(steric_R, batch)
        h_graph = global_add_pool(h, batch)
        base = self.node_mlp(torch.cat([h_graph, thermo, steric_graph], dim=-1))
        
        dist_R = (pos_R[row]-pos_R[col]).norm(dim=-1, keepdim=True)
        dist_P = (pos_P[row]-pos_P[col]).norm(dim=-1, keepdim=True)
        
        bo_R = self.pauling(z, edge_index, dist_R)
        bo_P = self.pauling(z, edge_index, dist_P)
        bo_TS = (1.0 - late_char_edge) * bo_R + late_char_edge * bo_P
        align = self.stereo(pos_R, pos_P, edge_index)
        hsab_E = self.hsab(hsab, edge_index)
        
        edge_feat = torch.cat([
            h[row], h[col], dist_R, dist_P, gate_score,
            bo_R, bo_P, bo_TS, bde, align, late_char_edge, hsab_E, pred_delta_D
        ], dim=-1)
        
        edge_out = self.edge_mlp(edge_feat) * gate_score
        barriers = global_add_pool(edge_out, batch[row])
        
        return barriers[:, :3] + base, barriers[:, 3:] + base, late_char

Model

In [43]:
class BEPGeoModel(nn.Module):
    def __init__(self, node_dim=27, hidden_dim=256, num_layers=4):
        super().__init__()
        self.node_emb = nn.Linear(node_dim, hidden_dim)
        self.diffusion = TSDiffusionModule(hidden_dim, hidden_dim)
        self.gating = SigmoidGating(hidden_dim)
        self.energy_predictor = PhysChemInformedPredictor(hidden_dim)

    def forward(self, data):
        h = self.node_emb(data.x)
        
        if self.training:
            struct_loss = self.diffusion.compute_loss(
                data.y_pos, h, data.edge_index, data.batch, data.pos, data.pos_P
            )
            # Robustness: Noise Injection
            noise_level = 0.1
            ts_pos_input = data.y_pos + torch.randn_like(data.y_pos) * noise_level
            
            dist_R = (data.pos[data.edge_index[0]] - data.pos[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS = (ts_pos_input[data.edge_index[0]] - ts_pos_input[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            pred_delta_D = dist_TS - dist_R

        else:
            struct_loss = 0.0
            generated_ts = self.diffusion.sample(
                data.pos, data.pos_P, h, data.edge_index, data.batch
            )
            ts_pos_input = generated_ts
            
            dist_R = (data.pos[data.edge_index[0]] - data.pos[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS_gen = (generated_ts[data.edge_index[0]] - generated_ts[data.edge_index[1]]).norm(dim=-1, keepdim=True)
            pred_delta_D = dist_TS_gen - dist_R

        # [수정] Gating Logits 계산
        gate_logits = self.gating(h, data.edge_index, data.pos)
        
        # [수정] 내부 연산용으로는 Sigmoid 적용 (0~1)
        gate_map = torch.sigmoid(gate_logits)
        
        thermo_sq = data.thermo_energy.squeeze(1) 
        pred_fwd, pred_rev, ts_char = self.energy_predictor(
            h, data.z, data.hsab, data.edge_index, data.pos, data.pos_P,
            gate_map, pred_delta_D, data.batch, thermo_sq, data.edge_bde
        )
        
        # [중요] Trainer에는 gate_logits(실수 범위)를 반환해야 BCEWithLogitsLoss 사용 가능
        return struct_loss, pred_fwd, pred_rev, gate_logits, ts_char, pred_delta_D

학습루프

In [44]:
class BEPTrainer:
    def __init__(self, model, optimizer, device, pretrain_mode=False):
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.pretrain_mode = pretrain_mode
        self.HARTREE_TO_KCAL = 627.5
        
    def train_step(self, batch):
        self.model.train()
        batch = batch.to(self.device)
        self.optimizer.zero_grad()
        
        # 모델 Forward (gate_logits를 받음)
        diff_loss, pred_fwd, pred_rev, gate_logits, ts_char, pred_d = self.model(batch)
        
        # [안전장치] NaN 체크 - 모델이 망가졌는지 확인
        if torch.isnan(diff_loss) or torch.isnan(gate_logits).any():
            print("⚠️ Warning: NaN detected in model output. Skipping this batch.")
            return 0.0, torch.tensor([0.0]), torch.tensor([0.0]) # 더미 리턴
        
        # 1. Structure Loss
        l_struct = diff_loss 
        
        if self.pretrain_mode:
            # Pre-training: 구조만 학습
            total_loss = l_struct
            target_fwd, pred_fwd = torch.tensor([0.0]), torch.tensor([0.0])
            
        else:
            # Fine-tuning: 전체 학습
            target_fwd = batch.y_fwd.squeeze(1) * self.HARTREE_TO_KCAL
            target_rev = batch.y_rev.squeeze(1) * self.HARTREE_TO_KCAL
            
            # Gating Loss [수정됨: Logits 기반 Loss 사용]
            # 수치적으로 훨씬 안정적이며 'input must be between 0 and 1' 에러가 발생하지 않음
            dist_R = (batch.pos[batch.edge_index[0]] - batch.pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS_true = (batch.y_pos[batch.edge_index[0]] - batch.y_pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
            
            target_gate = (torch.abs(dist_TS_true - dist_R) > 0.1).float()
            l_gate = F.binary_cross_entropy_with_logits(gate_logits, target_gate)
            
            # Energy Loss
            l_fwd = F.smooth_l1_loss(pred_fwd, target_fwd)
            l_rev = F.smooth_l1_loss(pred_rev, target_rev)
            
            # 가중치 적용
            w_struct = 2.0
            w_gate = 1.0
            w_energy = 0.05
            
            total_loss = (w_struct * l_struct) + (w_gate * l_gate) + (w_energy * (l_fwd + l_rev))
        
        # [안전장치] Total Loss NaN 체크
        if torch.isnan(total_loss):
            print("⚠️ Warning: Total Loss is NaN. Skipping backward.")
            return 0.0, torch.tensor([0.0]), torch.tensor([0.0])

        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        self.optimizer.step()
        
        return total_loss.item(), target_fwd, pred_fwd

In [45]:
def run_transfer_learning(t1x_path, rgd1_path, pretrain_epochs=20, finetune_epochs=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    # 모델 초기화 (한 번만 생성해서 계속 씀)
    model = BEPGeoModel(node_dim=27, hidden_dim=256, num_layers=4).to(device)
    
    # ---------------------------------------------------------
    # [Phase 1] Pre-training on Transition1x
    # 목적: 물리적으로 타당한 구조 생성 능력(Diffusion) 습득
    # ---------------------------------------------------------
    if t1x_path and os.path.exists(t1x_path):
        print("\n" + "="*50)
        print("🚀 Phase 1: Pre-training on Transition1x (Structure Only)")
        print("="*50)
        
        # T1x Dataset 로드 (메모리 절약을 위해 2만개만 사용 예시)
        t1x_dataset = Transition1xDataset(t1x_path, limit=20000)
        t1x_loader = DataLoader(t1x_dataset, batch_size=32, shuffle=True)
        
        # Pre-training용 Optimizer
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        
        # Trainer (Pretrain Mode ON)
        trainer = Trainer(model, optimizer, device, pretrain_mode=True)
        
        for epoch in range(pretrain_epochs):
            model.train()
            epoch_loss = 0
            pbar = tqdm(t1x_loader, desc=f"[Pretrain] Epoch {epoch+1}/{pretrain_epochs}")
            for batch in pbar:
                try:
                    loss, _, _ = trainer.train_step(batch)
                    epoch_loss += loss
                    pbar.set_postfix({'Diff_Loss': f"{loss:.4f}"})
                except Exception as e:
                    print(f"Error in batch: {e}")
                    continue
                    
        print("✅ Pre-training Completed. Model learned reaction paths.")
        
    else:
        print("⚠️ Transition1x path not found. Skipping Phase 1.")

    # ---------------------------------------------------------
    # [Phase 2] Fine-tuning on RGD1
    # 목적: 실제 타겟 활성화 에너지 및 정밀 구조 예측
    # ---------------------------------------------------------
    print("\n" + "="*50)
    print("🎯 Phase 2: Fine-tuning on RGD1 (Energy + Structure)")
    print("="*50)
    
    # RGD1 Dataset 로드 (기존 클래스 사용)
    rgd1_dataset = ReactionDataset(rgd1_path)
    rgd1_loader = DataLoader(rgd1_dataset, batch_size=32, shuffle=True)
    
    # Optimizer 재설정 (Fine-tuning은 LR을 낮춰서)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # Trainer (Pretrain Mode OFF - 이제 에너지도 학습)
    trainer = Trainer(model, optimizer, device, pretrain_mode=False)
    
    history = {'loss': []}
    real_E3 = {'H':[], 'G':[], 'E':[]}
    pred_E3 = {'H':[], 'G':[], 'E':[]}
    
    for epoch in range(finetune_epochs):
        model.train()
        epoch_loss = 0
        pbar = tqdm(rgd1_loader, desc=f"[Finetune] Epoch {epoch+1}/{finetune_epochs}")
        
        for batch in pbar:
            loss, y_true, y_pred = trainer.train_step(batch)
            epoch_loss += loss
            pbar.set_postfix({'Total_Loss': f"{loss:.4f}"})
            
            # 마지막 Epoch 결과 저장
            if epoch == finetune_epochs - 1:
                r = y_true.detach().cpu().numpy()
                p = y_pred.detach().cpu().numpy()
                real_E3['H'].extend(r.flatten())
                pred_E3['H'].extend(p.flatten())
        
        avg_loss = epoch_loss / len(rgd1_loader)
        history['loss'].append(avg_loss)
        scheduler.step(epoch_loss)

    return history, real_E3, pred_E3

In [46]:
if __name__ == "__main__":
    # 1. 공통 경로 설정
    # (사용자 환경에 맞게 수정된 경로 확인)
    base_dir = r"C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code"
    
    # 2. 파일 경로 생성
    t1x_h5_path = os.path.join(base_dir, "transition1x.h5")  # T1x 파일명
    rgd1_h5_path = os.path.join(base_dir, "RGD1_CHNO.h5")   # RGD1 파일명
    
    print(f"Checking files in: {base_dir}")
    print(f"T1x Path: {t1x_h5_path} (Exists: {os.path.exists(t1x_h5_path)})")
    print(f"RGD1 Path: {rgd1_h5_path} (Exists: {os.path.exists(rgd1_h5_path)})")

    # [핵심 수정] T1x와 RGD1 파일이 '모두' 존재해야만 실행
    if os.path.exists(t1x_h5_path) and os.path.exists(rgd1_h5_path):
        print("\n✅ 모든 데이터셋 파일이 확인되었습니다. 전이 학습을 시작합니다.")
        
        # run_transfer_learning 함수 호출
        hist, r_E3, p_E3 = run_transfer_learning(
            t1x_path=t1x_h5_path, 
            rgd1_path=rgd1_h5_path, 
            pretrain_epochs=100,  # 구조 학습 (T1x)
            finetune_epochs=200  # 에너지 학습 (RGD1)
        )
        
        # 4. 결과 시각화
        if hist:
            plt.figure(figsize=(16, 5))
            
            # Loss Plot
            plt.subplot(1, 4, 1)
            plt.plot(hist['loss'], 'k-')
            plt.title("Fine-tuning Loss")
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            
            # Parity Plots (H, G, E)
            titles, colors = ["Delta H", "Delta G", "Delta E"], ['r', 'g', 'b']
            keys = ['H', 'G', 'E']
            
            for i, k in enumerate(keys):
                if k in r_E3 and len(r_E3[k]) > 0:
                    plt.subplot(1, 4, i+2)
                    plt.scatter(r_E3[k], p_E3[k], alpha=0.3, color=colors[i], s=5)
                    
                    # Min/Max로 기준선 그리기
                    mx = max(max(r_E3[k]), max(p_E3[k]))
                    mn = min(min(r_E3[k]), min(p_E3[k]))
                    plt.plot([mn, mx], [mn, mx], 'k--')
                    
                    plt.title(titles[i])
                    plt.xlabel("DFT Calculation (kcal/mol)")
                    plt.ylabel("Model Prediction (kcal/mol)")
            
            plt.tight_layout()
            plt.show()
            
            # 성능 리포트 출력
            if 'print_evaluation_report' in globals():
                print_evaluation_report(r_E3, p_E3)
    
    # [에러 처리] 파일이 하나라도 없으면 실행 차단
    else:
        print("\n" + "="*50)
        print("❌ 실행 불가: 필수 데이터셋 파일이 누락되었습니다.")
        print("="*50)
        if not os.path.exists(t1x_h5_path):
            print(f" - [Missing] Transition1x 파일이 없습니다.\n   경로: {t1x_h5_path}")
        if not os.path.exists(rgd1_h5_path):
            print(f" - [Missing] RGD1 파일이 없습니다.\n   경로: {rgd1_h5_path}")
        print("="*50)

Checking files in: C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code
T1x Path: C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code\transition1x.h5 (Exists: True)
RGD1 Path: C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code\RGD1_CHNO.h5 (Exists: True)

✅ 모든 데이터셋 파일이 확인되었습니다. 전이 학습을 시작합니다.
Device: cpu

🚀 Phase 1: Pre-training on Transition1x (Structure Only)
Loading Transition1x Data from C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code\transition1x.h5 (Limit: 20000)...


Parsing T1x: 100%|███████████████████████████████████████████████████████████████████| 171/171 [00:16<00:00, 10.62it/s]


Loaded 10073 reactions for Pre-training.


[Pretrain] Epoch 68/100: 100%|█████████████████████████████████████| 315/315 [01:26<00:00,  3.63it/s, Diff_Loss=0.8087]


[Pretrain] Epoch 100/100: 100%|████████████████████████████████████| 315/315 [01:29<00:00,  3.51it/s, Diff_Loss=0.8017]


✅ Pre-training Completed. Model learned reaction paths.

🎯 Phase 2: Fine-tuning on RGD1 (Energy + Structure)


[Finetune] Epoch 1/200:  48%|████████████████▉                  | 2670/5531 [39:30<42:20,  1.13it/s, Total_Loss=4.3459]


KeyboardInterrupt: 

In [ ]:
import numpy as np

def print_evaluation_report(real_dict, pred_dict):
    """
    학습 결과(정답 vs 예측)를 받아서 MAE, RMSE, R2, MSE를 계산하고 출력합니다.
    """
    print("\n" + "="*75)
    print(f"   📊 FINAL PHYSICS-INFORMED MODEL PERFORMANCE REPORT (kcal/mol)")
    print("="*75)

    # 헤더 출력
    header = f"{'Energy Type':<15} | {'MAE':<10} | {'RMSE':<10} | {'MSE':<10} | {'R² Score':<10}"
    print(header)
    print("-" * 75)

    # H, G, E 각각에 대해 계산
    metrics = {}
    for key in ['H', 'G', 'E']:
        y_true = np.array(real_dict[key])
        y_pred = np.array(pred_dict[key])

        # 1. MAE (Mean Absolute Error)
        mae = np.mean(np.abs(y_true - y_pred))

        # 2. MSE (Mean Squared Error)
        mse = np.mean((y_true - y_pred)**2)

        # 3. RMSE (Root Mean Squared Error)
        rmse = np.sqrt(mse)

        # 4. R2 Score (Coefficient of Determination)
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2 = 1 - (ss_res / (ss_tot + 1e-8)) # 0으로 나누기 방지

        # 결과 저장
        metrics[key] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}

        # 행 출력
        label = f"Delta {key} (Δ{key})"
        print(f"{label:<15} | {mae:<10.4f} | {rmse:<10.4f} | {mse:<10.4f} | {r2:<10.4f}")

    print("="*75)

    # [화학적 해석]
    # 화학적 정확도(Chemical Accuracy)는 보통 1 kcal/mol 이내를 말합니다.
    avg_mae = np.mean([metrics[k]['MAE'] for k in ['H', 'G', 'E']])
    print(f"⭐ Average MAE: {avg_mae:.4f} kcal/mol")

    if avg_mae < 2.5:
        print("✅ 모델이 물리 법칙(BEP, Hammond, Pauling)을 매우 잘 학습했습니다!")
        print("   (일반적인 DFT 계산 오차가 2~3 kcal/mol 수준임을 감안하면 훌륭합니다.)")
    elif avg_mae < 5.0:
        print("⚠️ 학습이 잘 되었으나, 일부 복잡한 반응에서 오차가 있습니다.")
    else:
        print("❌ 아직 과소적합 상태이거나 물리적 제약 조건이 더 필요합니다.")
    print("="*75 + "\n")

# ==============================================================================
# [실행 방법] main 블록 안에서 train()이 끝난 후 호출하세요
# ==============================================================================
if __name__ == "__main__":
    # 위에서 이미 train을 실행해서 r_E3, p_E3가 있다고 가정합니다.
    # 만약 변수가 메모리에 없다면 다시 학습을 돌려야 합니다.

    if 'r_E3' in locals() and 'p_E3' in locals():
        print_evaluation_report(r_E3, p_E3)
    else:
        print("먼저 train() 함수를 실행하여 r_E3, p_E3 변수를 생성해주세요.")